In [1]:
import pickle
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import copy
import os
import numpy as np
from copy import copy
from torch.autograd import Variable
from torch.nn import functional as F
from collections import OrderedDict
from tqdm import tqdm

In [2]:
df = pd.read_pickle('tmall.pickle')
df.head()

,past_topic,future_topic,past_leaf
0,"[3, 0, 3, 20, 69, 69, 36, 22]","[32, 44, 0]","[[564180], [3405, 192788], [597499], [1101544]..."
1,"[47, 46, 7]","[0, 0, 0]","[[2076127], [2022749], [799008]]"
2,"[44, 0, 0, 44]","[0, 4, 0]","[[1931814], [354601, 477767, 477399, 472684, 4..."
3,"[39, 61, 3, 0, 16, 0, 3, 31, 45, 24]","[3, 0, 45]","[[1659033, 1643460, 1640173], [2298131], [5809..."
4,"[7, 50, 34, 28, 0, 15, 25]","[46, 6, 8]","[[790864], [2092319], [1464056, 1457409, 14467..."


In [3]:
with open('topic_items_to_emb-tmall.pkl','rb') as fp:
    embedding_map = pickle.load(fp)

In [4]:
with open('training_negative.pkl','rb') as fp:
    training_negative = pickle.load(fp)

In [5]:
x = []
y = []
all_zero = [0 for _ in range(256)]
for i in tqdm(range(int(len(df)*0.1))):
    row = df.iloc[i]
    past_id = row['past_topic']
    gt = row['future_topic']
    leaves = []
    for j in range(len(past_id)):
        key = tuple((past_id[j],tuple(row['past_leaf'][j])))
        leaves += list(embedding_map[key])
    
    for _ in range(10 - len(past_id)):
        leaves += all_zero
        
    temp = [72 for _ in range(10-len(past_id))]
    xs = []
    xs += past_id
    xs += temp
    xs += leaves
    candidate = list(copy(training_negative[i]))
    candidate += past_id
    for each in set(candidate):
        temp = copy(xs)
        temp.insert(11,each)
        x.append(temp)
        if each in gt:
            y.append(1)
        else:
            y.append(0)

100%|██████████| 41711/41711 [01:02<00:00, 669.62it/s] 


In [6]:
x = torch.tensor(x,dtype=torch.float)
y = torch.tensor(y,dtype=torch.float)
x = x.to('cuda:0')
y = y.to('cuda:0')

In [7]:
config = {
    'embedding_dim': 64,
    'num_cate': 72,
    'vae' : 256,
    'dim1' : 1024,
    'dim2' : 256
}

In [8]:
class user_preference_estimator(torch.nn.Module):
    def __init__(self, config):
        super(user_preference_estimator, self).__init__()
        
        self.embedding_dim = config['embedding_dim']
        self.num_cateogry = config['num_cate'] + 1
        self.vae = config['vae']
        self.fc1_in_dim = config['embedding_dim']  * 10 * 2 + config['embedding_dim']
        self.fc2_in_dim = config['dim1']
        self.fc2_out_dim = config['dim2']

        
        self.embedding = nn.Embedding(self.num_cateogry,self.embedding_dim)
        self.vaeencoder = torch.nn.Linear(self.vae,self.embedding_dim)
        self.fc1 = torch.nn.Linear(self.fc1_in_dim, self.fc2_in_dim)
        self.fc2 = torch.nn.Linear(self.fc2_in_dim, self.fc2_out_dim)
        self.linear_out = torch.nn.Linear(self.fc2_out_dim, 1)
    
    
    def forward(self, x):
        #First ten entries are categories
        category = Variable(x[:, 0:11], requires_grad=False)
        leaf = Variable(x[:,11:], requires_grad=False)
        leaf = leaf.view(-1,10,self.vae)
        out = self.embedding(category.long())
        out = out.view(-1,self.embedding_dim*11)
        out2 = self.vaeencoder(leaf)
        out2 = out2.view(-1,self.embedding_dim*10)
        out = torch.cat((out, out2), 1)
        out = self.fc1(out)
        out = F.relu(out)

        out = self.fc2(out)
        out = F.relu(out)

        out = self.linear_out(out)
        return out
        
        

In [9]:
estimator = user_preference_estimator(config)
estimator.to('cuda:0')

user_preference_estimator(
  (embedding): Embedding(73, 64)
  (vaeencoder): Linear(in_features=256, out_features=64, bias=True)
  (fc1): Linear(in_features=1344, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=256, bias=True)
  (linear_out): Linear(in_features=256, out_features=1, bias=True)
)

In [1]:
optimizer = torch.optim.Adam(estimator.parameters(), lr = 0.001)
for _ in range(5):
    for i in tqdm(range(len(x))):
        losses_q = []
        y_pred = estimator.forward(x[i:i+1])
        loss_q = F.mse_loss(y_pred, y[i:i+1].view(-1, 1))
        losses_q.append(loss_q)
        losses_q = torch.stack(losses_q).mean(0)

        optimizer.zero_grad()
        with torch.autograd.set_detect_anomaly(True):
            losses_q.clamp(min=1e-6)
            losses_q.backward()
        optimizer.step()
    torch.save(estimator.state_dict(),"final.pkl")


NameError: name 'torch' is not defined

In [11]:
with open('testing_negative.pkl','rb') as fp:
    testing_negative = pickle.load(fp)

In [14]:
xt = []
yt = []
all_zero = [0 for _ in range(256)]
cand = []
offset = int(len(df)*0.9)
for i in tqdm(range(int(len(df)*0.9),int(0.91*len(df)))):
    row = df.iloc[i]
    past_id = row['past_topic']
    gt = row['future_topic']
    leaves = []
    for j in range(len(past_id)):
        key = tuple((past_id[j],tuple(row['past_leaf'][j])))
        leaves += list(embedding_map[key])
    
    for _ in range(10 - len(past_id)):
        leaves += all_zero
        
    temp = [72 for _ in range(10-len(past_id))]
    xs = []
    xs += past_id
    xs += temp
    xs += leaves
    candidate = list(copy(testing_negative[i-offset]))
    candidate += past_id
    candidate = list(set(candidate))
    cx = []
    for each in candidate:
        temp = copy(xs)
        temp.insert(11,each)
        cx.append(temp)
    xt.append(cx)
    yt.append(gt)
    cand.append(candidate)

100%|██████████| 4171/4171 [00:02<00:00, 1737.18it/s]


In [15]:
def ComputeScore(ret,gt,t,show=True):
    tp = 0
    fp = 0
    fn = 0
    h = 0
    for i in range(len(ret)):
        ht = 0
        for each in ret[i][0:t]:
            if each in gt[i]:
                tp += 1
                ht = 1
            else:
                fp += 1
        for each in gt[i]:
            if each not in ret[i]:
                fn += 1
        h += ht
    p = tp/(tp+fp)
    r = tp/(tp+fn)
    f = 2*p*r/(p+r)
    if show:
        print("HR@" + str(t) + "       :" + str(h/len(ret)))
        print("Precision@" + str(t) + ":" + str(p))
        print("Recall@" + str(t) + "   :" + str(r))
        print("F1@" + str(t) + "       :" + str(f))
        print("*************************************************************")
    return h/len(ret)

In [35]:
def TestModel(m,x,gt,cand):
    ret = []
    for i in tqdm(range(len(x))):
        xc = x[i]
        xc = torch.tensor(xc,dtype=torch.float)
        out = m(xc)
        out = out.detach().numpy().flatten()
        sorting = np.argsort(-1 * out)
        temp = []
        for each in sorting:
            temp.append(cand[i][each])
        ret.append(temp)
    for t in [1,3,5,7,10]:
        ComputeScore(ret,gt,t)

In [37]:
model = user_preference_estimator(config)
model.load_state_dict(torch.load("final.pkl"))
TestModel(model,xt,yt,cand)

  0%|          | 0/4171 [00:00<?, ?it/s]

100%|██████████| 4171/4171 [00:15<00:00, 263.92it/s]

HR@1       :0.2155358427235675
Precision@1:0.2155358427235675
Recall@1   :0.15686616646309545
F1@1       :0.18157947889315287
*************************************************************
HR@3       :0.40517861424118917
Precision@3:0.14544873331734995
Recall@3   :0.2736019242333133
F1@3       :0.18992955909209494
*************************************************************
HR@5       :0.635578997842244
Precision@5:0.15823543514744665
Recall@5   :0.4058042302016724
F1@5       :0.2276882740538862
*************************************************************
HR@7       :0.8010069527691202
Precision@7:0.16566770558619037
Recall@7   :0.5002585582790361
F1@7       :0.24890649925384659
*************************************************************
HR@10       :0.888995444737473
Precision@10:0.14799808199472547
Recall@10   :0.5609268514311676
F1@10       :0.23420278858010052
*************************************************************
